<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/git_teste_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importa pacotes que serão usados

In [27]:
# !rm -rf cassandra*
# !rm -rf testes

In [28]:
import pandas as pd
import os
import requests

def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

def instala_cloc():
  try: 
    comando = !sudo apt-get install cloc
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')

def clona_repositorio(url):
  try:
    comando = f'git clone {url}'
    print(f'Clonando o repositório {url}')
    print(comando)
    os.system(comando)
    print('concluido com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')
  
def get_primeiro_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit 
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | tail -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_commit_tag(tag):
  try:
    os.environ['TAG'] = tag
    comando = !cd cassandra && git rev-list -n 1 $TAG
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_ultimo_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit 
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | head -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def cria_pasta_n_commit(repositorio, pasta_n_commit):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Cria uma nova pasta para referenciar apenas os dados do 1o commit
    comando = !cp -R $REPOSITORIO_ORIGINAL $REPOSITORIO_INICIAL
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def checkout_n_commit(pasta_n_commit, n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    os.environ['PRIMEIRO_COMMIT'] = n_commit
    # git checkout $( git rev-list --all | tail -n 1 )
    comando1 = "cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT"
    os.system(comando1)
    print(f'Checkout do repositorio no commit {n_commit} realizado com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_info_repositorio(pasta_n_commit):
  try: 
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Quantidade de arquivos na pasta cassandra
    qtd_arquivos = !find $REPOSITORIO_INICIAL -type f | wc -l
    # Quantidade de commits do repositorio cassandra
    qtd_commits = !cd $REPOSITORIO_INICIAL && git log --oneline | grep -c .
    # Tamanho (MB) da pasta cassandra
    tamanho_pasta_mb = !du -hs --block-size=1M $REPOSITORIO_INICIAL | cut -f1
    # Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto cassandra)
    qtd_colaboradores_commits = !cd $REPOSITORIO_INICIAL && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l
    # Data do primeiro commit do projeto cassandra
    data_primeiro_commit =!cd $REPOSITORIO_INICIAL && git log --reverse --pretty=format:%ad --date=short | head -1
    # Data do ultimo commit do projeto cassandra
    data_ultimo_commit = !cd $REPOSITORIO_INICIAL && git log -1 --pretty=format:%ad --date=short

    dict_info_repo = {
      'nome':pasta_n_commit,
      'qtd_arquivos':qtd_arquivos,
      'qtd_commits':qtd_commits,
      'tamanho_pasta_mb':tamanho_pasta_mb,
      'qtd_colaboradores_commits':qtd_colaboradores_commits,
      'data_primeiro_commit':data_primeiro_commit,
      'data_ultimo_commit':data_ultimo_commit,
    }

    df_info_repo = pd.DataFrame(dict_info_repo)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return df_info_repo

def gera_df_dados(pasta_n_commit):
  lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()
  l_temp = []
  vl_temp = []
  os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
  try:
    # Detalha os tipos de arquivos que aparecem no repositório do cassandra
    relacao_linguagens = !cd $REPOSITORIO_INICIAL && cloc .
    #relacao_linguagens[checa_indice_language(relacao_linguagens):]
    linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
    for linha in linguagens_encontradas:
      elementos = linha.split(' ')
      novos_elementos = []
      linguagem_encontrada = []
      valores_da_linguagem = []
      for item in elementos:
        if item != '':
          novos_elementos.append(item)
      linguagem_encontrada.append(novos_elementos[:-4])
      valores_da_linguagem.append(novos_elementos[-4:])
      l_temp = l_temp + linguagem_encontrada
      vl_temp = vl_temp + valores_da_linguagem

    l_temp = l_temp[:-3]
    vl_temp = vl_temp[:-3]
    for i,each in enumerate(vl_temp):
      lista_files.append(each[0])
      lista_blank.append(each[1]) 
      lista_comment.append(each[2]) 
      lista_code.append(each[3])  
  except Exception as e:
    print(f'Erro: {str(e)}')
  try:
    dict_dados_linguagens = {
    'linguagens':l_temp, 
    'files':lista_files, 
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
    }
    df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
    df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
    df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))	
    df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))	
    df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))
  except Exception as e2:
    print(f'Erro: {str(e2)}')

  return df_dados_linguagens

def gera_df_dados_repositorios(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'descricao':l_cf,
        'dados_c1':c_c1,
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def gera_df_dados_repositorios2(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def download_designite():
  try:
    url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
    filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
    response = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f'{filename} downloaded successfully!')
  except Excetpion as e:
    print(f'Erro: {str(e)}')

def run_designite(tag, pasta_testes):
  try:
    os.environ['TESTES'] = pasta_testes
    os.environ['CASSANDRA'] = tag + '/src/java/org/apache/cassandra'
    cria_pasta_testes = !mkdir $TESTES
    print(cria_pasta_testes)
    executa_designite = !java -jar DesigniteJava.jar -i $CASSANDRA -o $TESTES
    print(executa_designite)
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_design_smells(pasta_testes, lista):
  path_file = pasta_testes + '/' + 'designCodeSmells.csv'
  df_ds = pd.read_csv(path_file)
  elemento = pasta_testes,df_ds
  lista.append(elemento)
  return df_ds

In [29]:
instala_cloc()

['', 'Reading package lists... 0%', '', 'Reading package lists... 0%', '', 'Reading package lists... 0%', '', 'Reading package lists... 4%', '', 'Reading package lists... 4%', '', 'Reading package lists... 4%', '', 'Reading package lists... 4%', '', 'Reading package lists... 45%', '', 'Reading package lists... 45%', '', 'Reading package lists... 46%', '', 'Reading package lists... 46%', '', 'Reading package lists... 48%', '', 'Reading package lists... 56%', '', 'Reading package lists... 56%', '', 'Reading package lists... 64%', '', 'Reading package lists... 64%', '', 'Reading package lists... 69%', '', 'Reading package lists... 69%', '', 'Reading package lists... 69%', '', 'Reading package lists... 69%', '', 'Reading package lists... 70%', '', 'Reading package lists... 70%', '', 'Reading package lists... 70%', '', 'Reading package lists... 70%', '', 'Reading package lists... 79%', '', 'Reading package lists... 79%', '', 'Reading package lists... 86%', '', 'Reading package lists... 86%'

# Clona repositorio principal

In [30]:
clona_repositorio(url='https://github.com/apache/cassandra.git')
print(f"Primeiro commit do cassandra: {get_primeiro_commit('cassandra')}")
print(f"Ultimo commit do cassandra: {get_ultimo_commit('cassandra')}")

Clonando o repositório https://github.com/apache/cassandra.git
git clone https://github.com/apache/cassandra.git
concluido com sucesso!
Primeiro commit do cassandra: ['1f91e99223b0d1b7ed8390400d4a06ac08e4aa85']
Ultimo commit do cassandra: ['4e3447ae1727a0d01c9d81792acbdeeef94d86d2']


#Testes do 1o. commit e último commit do cassandra

## Cria um repositorio apontando para o 1o. commit

In [31]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit='cassandra_c1')
# git checkout $( git rev-list --all | tail -n 1 )
#!cd $REPOSITORIO_INICIAL && git checkout 1f91e99223b0d1b7ed8390400d4a06ac08e4aa85
primeiro_commit = get_primeiro_commit(repositorio='cassandra')
checkout_n_commit('cassandra_c1', primeiro_commit[0])
df_info_repo = gera_df_info_repositorio(pasta_n_commit='cassandra_c1')
df_dados_linguagens = gera_df_dados(pasta_n_commit='cassandra_c1')

Checkout do repositorio no commit 1f91e99223b0d1b7ed8390400d4a06ac08e4aa85 realizado com sucesso!


## Cria um repositório apontando para o último commit

In [32]:
# cria pasta o primeiro/ultimo commit
cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit='cassandra_cf')
# pega o primero/ultimo commit
ultimo_commit = get_ultimo_commit(repositorio='cassandra')
# faz o checkout para primeiro/ultimo commit
checkout_n_commit('cassandra_cf', ultimo_commit[0])
# gera dataframe de info do repositorio
df_info_repo_cf = gera_df_info_repositorio(pasta_n_commit='cassandra_cf')
# gera dataframe de dados linguagem do repositorio
df_dados_linguagens_cf = gera_df_dados(pasta_n_commit='cassandra_cf')

Checkout do repositorio no commit 4e3447ae1727a0d01c9d81792acbdeeef94d86d2 realizado com sucesso!


## Gera dataframe com os dados dos repositorios

In [33]:
df_repo = gera_df_dados_repositorios(df_info_repo, df_info_repo_cf)
df_repo

,descricao,dados_c1,dados_cf
0,nome,cassandra_c1,cassandra_cf
1,qtd_arquivos,369,5186
2,qtd_commits,1,28432
3,tamanho_pasta_mb,408,487
4,qtd_colaboradores_commits,1,578
5,data_primeiro_commit,2009-03-02,2009-03-02
6,data_ultimo_commit,2009-03-02,2023-03-10


In [34]:
lista_descricao_repo = df_repo['descricao'].to_list()

In [35]:
df_dados_linguagens_cf

,linguagens,files,blank,comment,code
0,[Java],3877,116507,146654,613993
1,[YAML],45,767,2563,330789
2,[AsciiDoc],109,5592,11,20115
3,[Python],31,2460,3163,8332
4,[XML],33,201,478,3740
5,[Markdown],13,773,0,3387
6,[SVG],17,197,1399,2937
7,"[Bourne, Shell]",82,369,987,1970
8,"[ANTLR, Grammar]",3,267,288,1897
9,[Ant],3,185,233,1787


In [36]:
lista_descricao_dados_linguagens_cf = df_dados_linguagens_cf.columns.tolist()
lista_descricao_dados_linguagens_cf

['linguagens', 'files', 'blank', 'comment', 'code']

In [37]:
df_repo_linguagens = gera_df_dados_repositorios(df_dados_linguagens_cf, df_dados_linguagens_cf)
df_repo_linguagens

,descricao,dados_c1,dados_cf
0,linguagens,[Java],[Java]
1,files,3877,3877
2,blank,116507,116507
3,comment,146654,146654
4,code,613993,613993


In [38]:
# Lista as tags registradas do repositorio cassandra
lista_tags = !cd cassandra && git tag

In [39]:
# Pega o primeiro commit da tag alvo
!cd cassandra && git rev-list -n 1 cassandra-3.0.0

96f407bce56b98cd824d18e32ee012dbb99a0286


# Instala o Designite Java

In [40]:
# testes do Designite
download_designite()

DesigniteJava.jar downloaded successfully!


# Testes com as tags do cassandra

In [41]:
# Cria listas para armazenar os dataframes de cada tag
lista_df_info_repo = []
lista_df_dados_linguagens =[]
lista_df_repo = []
lista_df_repo_linguagens = []
lista_df_designite = []

# Lista de tags das analises do repositorio do cassandra
lista_my_tags = ['cassandra-3.0.0', 'cassandra-3.1', 'cassandra-3.2',
 'cassandra-3.3',
 'cassandra-3.4',
 'cassandra-3.5',
 'cassandra-3.6',
 'cassandra-3.7',
 'cassandra-3.8',
 'cassandra-3.9',
 'cassandra-3.10',
 'cassandra-3.11.0',
 'cassandra-3.11.11']

for my_tag in lista_my_tags: 
  print(f'Analisando a tag: {my_tag}')
  # 1. Cria uma pasta com o nome da tag alvo
  cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit=my_tag)
  # 2. Pega o primeiro_commit da tag alvo
  primeiro_commit = get_commit_tag(tag=my_tag)
  # 3. Faz o checkout do repositorio para a tag alvo
  checkout_n_commit(my_tag, primeiro_commit[0])
  # 4. Gera os dataframes de informacoes do checkout e linguagens do checkout
  df_info_repo = gera_df_info_repositorio(pasta_n_commit=my_tag)
  df_dados_linguagens = gera_df_dados(pasta_n_commit=my_tag)
  print(f'Análise da tag: {my_tag} concluída com sucesso!')

  print(f'Gera o dataframe de dados do repo {my_tag}')
  df_repo = gera_df_dados_repositorios2(df_info_repo, df_info_repo)
  lista_df_repo.append(df_repo)

  print(f'Gera o dataframe de dados das linguagens do repo {my_tag}')
  df_repo_linguagens_final = gera_df_dados_repositorios2(df_dados_linguagens, df_dados_linguagens)
  lista_df_repo_linguagens.append(df_repo_linguagens_final)

  # 5. Registra a tupla (my_tag, df) das informacoes e linguagens do checkout
  lista_df_info_repo.append((my_tag,df_info_repo))
  lista_df_dados_linguagens.append((my_tag,df_dados_linguagens))

  # 6. Faz a analise de design smells usando o designite
  pasta_testes = 'testes' + '_' +  my_tag
  run_designite(my_tag, pasta_testes)
  gera_df_design_smells(pasta_testes, lista_df_designite)

Analisando a tag: cassandra-3.0.0
Checkout do repositorio no commit 96f407bce56b98cd824d18e32ee012dbb99a0286 realizado com sucesso!
Análise da tag: cassandra-3.0.0 concluída com sucesso!
Gera o dataframe de dados do repo cassandra-3.0.0
Gera o dataframe de dados das linguagens do repo cassandra-3.0.0
[]
['Parsing the source code ...', 'Resolving symbols...', 'Extracting metrics...', 'Extracting code smells...', 'Exception while parsing literal number (during Magic Number detection). For input string: "c6a4a7935bd1e995"', 'Exception while parsing literal number (during Magic Number detection). For input string: "ff51afd7ed558ccd"', 'Exception while parsing literal number (during Magic Number detection). For input string: "c4ceb9fe1a85ec53"', 'Exception while parsing literal number (during Magic Number detection). For input string: "87c37b91114253d5"', 'Exception while parsing literal number (during Magic Number detection). For input string: "c000000000000000"', 'Exception while parsing 

In [42]:
dict_descricao1 = {
    'descricao':lista_descricao_repo
}
df_descricao1 = pd.DataFrame(dict_descricao1)

dict_descricao2 = {
    'descricao':lista_descricao_dados_linguagens_cf
}

df_descricao2 = pd.DataFrame(dict_descricao2)

In [43]:
# Faz um merge dos dataframes gerados
# df_final = df1 + df2 + ... + dfN
lista_df_repo.insert(0, df_descricao1)
df_dados_final = pd.concat(lista_df_repo, axis=1)

lista_df_repo_linguagens.insert(0, df_descricao2)
df_linguagens_final = pd.concat(lista_df_repo_linguagens, axis=1)

In [44]:
df_dados_final.columns = ['descricao', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13']
df_dados_final

,descricao,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13
0,nome,cassandra-3.0.0,cassandra-3.1,cassandra-3.2,cassandra-3.3,cassandra-3.4,cassandra-3.5,cassandra-3.6,cassandra-3.7,cassandra-3.8,cassandra-3.9,cassandra-3.10,cassandra-3.11.0,cassandra-3.11.11
1,qtd_arquivos,2406,2419,2467,2476,2644,2782,2828,2896,3063,3063,3197,3233,3605
2,qtd_commits,18441,18760,19538,19763,20083,20266,20602,20769,21345,21347,22057,22438,24000
3,tamanho_pasta_mb,497,497,498,498,504,506,508,509,511,511,514,515,494
4,qtd_colaboradores_commits,201,208,220,225,234,238,250,257,279,279,304,321,405
5,data_primeiro_commit,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02
6,data_ultimo_commit,2015-11-06,2015-12-04,2016-01-07,2016-02-03,2016-02-16,2016-04-07,2016-05-30,2016-06-06,2016-09-26,2016-09-26,2017-01-30,2017-06-19,2021-07-25


In [45]:
df_linguagens_final.columns = ['descricao', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13']
df_linguagens_final

,descricao,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13
0,linguagens,[Java],[Java],[Java],[Java],[Java],[Java],[Java],[Java],[Java],[Java],[Java],[Java],[Java]
1,files,1763,1771,1817,1826,1951,1965,2006,2010,2037,2037,2143,2158,2353
2,blank,58813,59265,60348,60759,65622,66305,68023,68190,69763,69763,73615,74531,82082
3,comment,67779,68230,69407,69751,73970,74501,76400,76839,80385,80385,85296,86203,93702
4,code,316059,318674,325126,327066,348433,351590,360937,361994,369153,369153,388942,392843,432977


In [46]:
# cria uma nova linha
#nova_linha = pd.DataFrame({'A': [0], 'B': [0]})

# concatena o novo dataframe com o dataframe original usando o método concat()
#df = pd.concat([nova_linha, df]).reset_index(drop=True)